In [ ]:
!pip install unidecode openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 8.0 MB/s eta 0:00:00


In [ ]:
import os
import re
import glob
import html
import pandas as pd
from unidecode import unidecode

FOLDER_PATH = "/content/extracted_RS_raw_data"
FILE_PATTERN = "*.csv"
OUTPUT_CSV = "/content/cleaned_merged_data.csv"

TITLE_COL = "title"
SELFTEXT_COL = "selftext"
CREATED_UTC_COL = "created_utc"

# 1. Text cleaning
def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text_stripped = text.strip().lower()
    if text_stripped in ["[deleted]", "[removed]", ""]:
        return ""
    text = html.unescape(text)
    text = unidecode(text)
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"[^a-zA-Z0-9\s\.\,\!\?\']", " ", text)
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    return text

# 2. Read and merge files
def read_and_merge(folder_path: str, file_pattern: str) -> pd.DataFrame:
    files = sorted(glob.glob(os.path.join(folder_path, file_pattern)))
    if not files:
        print(f"[WARNING] No files found in: {folder_path}/{file_pattern}")
        return pd.DataFrame()

    all_dfs = []
    for file_path in files:
        print(f"[INFO] Reading: {file_path}")
        if file_path.endswith(".csv"):
            df = pd.read_csv(file_path, encoding="utf-8", keep_default_na=False)
        else:
            df = pd.read_excel(file_path, engine="openpyxl", keep_default_na=False)
        all_dfs.append(df)

    if not all_dfs:
        return pd.DataFrame()

    merged_df = pd.concat(all_dfs, ignore_index=True)
    print(f"[INFO] Successfully merged {len(files)} file(s). Total rows: {len(merged_df)}")
    return merged_df

# 3. Main process: merge, clean, save
df_merged = read_and_merge(FOLDER_PATH, FILE_PATTERN)

if df_merged.empty:
    print("[ERROR] No data to process. Exiting.")
else:
    # a) Add date columns
    if CREATED_UTC_COL in df_merged.columns:
        df_merged["created_dt"] = pd.to_datetime(df_merged[CREATED_UTC_COL], unit="s", errors="coerce")
        df_merged["month"] = df_merged["created_dt"].dt.strftime("%Y-%m")
    else:
        print(f"[WARNING] Column '{CREATED_UTC_COL}' not found. Unable to create 'month'.")
        df_merged["created_dt"] = None
        df_merged["month"] = None

    # b) Remove rows where both title and selftext are useless
    def is_useless(text):
        return str(text).strip().lower() in ["[removed]", "[deleted]", ""]

    df_merged = df_merged[
        ~(
            df_merged[TITLE_COL].apply(is_useless) &
            df_merged[SELFTEXT_COL].apply(is_useless)
        )
    ].reset_index(drop=True)

    # ✅ NEW: Clean [deleted]/[removed] BEFORE merging
    def clean_raw_field(text):
        if str(text).strip().lower() in ["[deleted]", "[removed]"]:
            return ""
        return text

    # c) Combine + clean
    df_merged["combined_text"] = (
        df_merged[TITLE_COL].apply(clean_raw_field).fillna("") + " " +
        df_merged[SELFTEXT_COL].apply(clean_raw_field).fillna("")
    )

    df_merged["clean_text"] = df_merged["combined_text"].apply(clean_text)

    # d) Drop rows with empty clean_text
    before_drop = len(df_merged)
    df_merged = df_merged[df_merged["clean_text"].str.strip() != ""]
    after_drop = len(df_merged)
    print(f"[INFO] Dropped {before_drop - after_drop} empty rows. Remaining rows: {after_drop}")

    # e) Save cleaned file
    output_path = os.path.join(FOLDER_PATH, OUTPUT_CSV)
    df_merged.to_csv(output_path, index=False, encoding="utf-8")
    print(f"[DONE] Cleaned data saved to: {output_path}")

    print("[INFO] DataFrame columns:", df_merged.columns.tolist())
    print("[INFO] Head of the DataFrame:")
    print(df_merged.head(3))


[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2022-12_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-01_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-02_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-03_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-04_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-05_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-06_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-07_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-08_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-09_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-10_filtered.csv
[INFO] Reading: /content/extracted_RS_raw_data/extracted_RS_2023-11_filtered.csv
[INFO] Successfully merged 1

# 新段落